#Download the Dataset

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
import opendatasets as om

import pandas
#download given dataset for the original problem
od.download(
    "https://www.kaggle.com/datasets/kasikrit/att-database-of-faces")
#download dataset for face-nonface
om.download(
    "https://www.kaggle.com/datasets/arsanymousafathy/facenonfacedata")

In [ ]:
import os
import numpy as np
import cv2
from PIL import Image
from scipy.spatial.distance import cdist
from sklearn.decomposition import KernelPCA
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import time
from PIL import Image as im
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import seaborn as sns

#Functions

##Get ID

In [ ]:
def get_id(s):
    id = s[-2:]
    if id[0] == 's':
      id = id[-1:]
    return id

##50-50 Split (Even and odd)

In [ ]:
def split_even_odd(data_matrix, label_vector):
  testing_set=data_matrix[::2] #slicing notation: [start:stop:step]
  training_set=data_matrix[1::2]
  testing_labels=label_vector[::2]
  training_labels=label_vector[1::2]
  return training_set, testing_set, training_labels, testing_labels 

##Ratio Splitting

In [ ]:
def split(data_matrix, label_vector, train_portion, n_classes, n_perclass):
  test_portion = 1-train_portion
  data_size = data_matrix.shape[0]
  feature_size = data_matrix.shape[1]
  training_data_size = int(train_portion * data_size)
  test_data_size = int(test_portion * data_size)
  n_trainperclass = int(train_portion * n_perclass)
  n_testperclass = int(test_portion * n_perclass)
  testing_set = np.empty([test_data_size, feature_size])
  training_set = np.empty([training_data_size, feature_size])
  testing_labels=np.empty([test_data_size,1])
  training_labels=np.empty([training_data_size,1])
  for i in range(n_classes):
    training_set[i*n_trainperclass:(i*n_trainperclass)+n_trainperclass, :] = data_matrix[i*n_perclass:(i*n_perclass)+n_trainperclass, :]
    testing_set[i*n_testperclass:(i*n_testperclass)+n_testperclass, :] = data_matrix[(i*n_perclass)+n_trainperclass:(i*n_perclass)+n_perclass, :]
    testing_labels[i*n_testperclass:(i*n_testperclass)+n_testperclass, :] = label_vector[(i*n_perclass)+n_trainperclass:(i*n_perclass)+n_perclass, :]
    training_labels[i*n_trainperclass:(i*n_trainperclass)+n_trainperclass] = label_vector[i*n_perclass:(i*n_perclass)+n_trainperclass]
  return training_set, testing_set, training_labels, testing_labels 

##Splitting for faces vs non-faces

In [ ]:
def split_nonfaces(data_matrix, label_vector):
  training_set, testing_set_d, training_labels, testing_labels_d = split_even_odd(data_matrix, label_vector)
  testing_set=np.empty([100,10304])
  testing_labels=np.empty([100,1])
  testing_set[0:50,:]=testing_set_d[0:50,:]
  testing_labels[0:50,:]=testing_labels_d[0:50,:]
  testing_set[50:100,:]=testing_set_d[-50:,:]
  testing_labels[50:100,:]=testing_labels_d[-50:,:]
  return training_set, testing_set, training_labels, testing_labels 

##K-Nearest Neighbour 

In [ ]:
def knn (number_of_classes, training_labels, euc_dist_matrix, k):
  test_data_size = euc_dist_matrix.shape[0]
  classify_vectorknn=np.empty([test_data_size,1])
  for j in range(euc_dist_matrix.shape[0]):
    voting = np.zeros((number_of_classes,2))
    sorted_indices = np.argsort(euc_dist_matrix[j])     # Get indices that would sort the array in ascending order
    least_k_indices = sorted_indices[:k]
    for v in range(k):
      id_index = least_k_indices[v]
      id = training_labels[id_index] - 1
      id = int(id)
      voting[id,0] = voting[id,0] + 1
      voting[id,1] = voting[id,1] + (euc_dist_matrix[j,id_index])
    max_value = np.amax(voting[:,0], axis=0)
    indices = np.where(voting[:,0] == max_value)
    indices = np.array(indices)
    indices=indices[0,:]
    min = indices[0]
    for ind in range(len(indices)):
      x1 = int(voting[indices[ind],1])
      x2 = int(voting[min,1])
      if x1 < x2:
        min = indices[ind]
    winner = min + 1
    classify_vectorknn[j] = winner
  return classify_vectorknn

##PCA

In [ ]:
def pca(D,label_vector, alpha, train_portion, n_classes, n_perclass, k_knn, normal_split):
  # Splitting the dataset
  if normal_split ==False :
    training_set, testing_set, training_labels, testing_labels = split_nonfaces(D, label_vector)
  elif train_portion == 0.5:
    training_set, testing_set, training_labels, testing_labels = split_even_odd(D, label_vector)
  else:
    training_set, testing_set, training_labels, testing_labels = split(D, label_vector, train_portion, n_classes, n_perclass)  
  training_data_size = training_labels.shape[0]
  test_data_size = testing_labels.shape[0]
  n_trainperclass = int(train_portion * n_perclass)
  #Mean Vector 
  mean_vector = np.mean(training_set, axis=0)
  #Centering the data
  Z_training = training_set - mean_vector 
  Z_training_transpose = np.transpose(Z_training)
  #Covariance Matrix
  cov_mat = (Z_training_transpose@Z_training)/Z_training.shape[0] #the symbol @ stands for matrix multiplication.
  #Eigen Values and Eigen Vectors
  eig_values, eig_vectors = np.linalg.eigh(cov_mat)
  indices = eig_values.argsort()[::-1]  
  eig_values_vector = eig_values[indices]
  eig_values = np.diag(eig_values_vector)
  eig_vectors = eig_vectors[:,indices]
  if isinstance(alpha, float):
    alpha=[alpha]
  if isinstance(k_knn, (int, float)):
    k_knn=[k_knn]
  n = len(alpha) #number of different values of alpha
  r_alpha = [0 for _ in range(n)]
  sum = 0
  for i in range(eig_values_vector.shape[0]):
    sum = sum + (eig_values_vector[i]/np.sum(eig_values_vector))
    for j in range(n):
      if sum >= alpha[j]:
        if r_alpha[j] == 0:
          r_alpha[j] = i
      else:
        break
    if r_alpha[n-1] != 0:
      break
  accuracy = np.empty([n, len(k_knn)])
  #Projection
  for a in range(n):
    P = eig_vectors[:,0:r_alpha[a]]
    projected_training_matrix = np.dot(Z_training, P)
    Z_testing = testing_set - mean_vector
    projected_testing_matrix = np.dot(Z_testing, P)
    euc_dist = cdist(projected_testing_matrix, projected_training_matrix)
    for kk in range(len(k_knn)):
      classify_vector = knn(n_classes, training_labels, euc_dist, k_knn[kk])
      y = np.count_nonzero(testing_labels-classify_vector)
      cm = confusion_matrix(testing_labels, classify_vector, labels=confusion_labels)
      accuracy[a, kk] = 1-y/test_data_size
      if n_classes == 2 or normal_split == False:
        for o in range(test_data_size):
          array = np.reshape(testing_set[o,:], (112,92))
          data = im.fromarray(array)
          plt.imshow(data)
          plt.axis('off')  
          plt.show()
          if classify_vector[o] == 1:
            print("This is a Non-face image")
          else:
            print("This is a face image")
     # else:
       # images_display(test_data_size,testing_set,training_labels,classify_vector,training_set)
  return accuracy, cm

##Kernel PCA

In [ ]:
def kernel_pca(D, label_vector, train_portion, n_classes, n_perclass, components):
   # Splitting the dataset
  if train_portion == 0.5:
    training_set, testing_set, training_labels, testing_labels = split_even_odd(D, label_vector)
  else:
    training_set, testing_set, training_labels, testing_labels = split(D, label_vector, train_portion, n_classes, n_perclass)
  training_data_size = training_labels.shape[0]
  test_data_size = testing_labels.shape[0]
  n_trainperclass = int(train_portion * n_perclass)
  #Mean Vector 
  mean_vector = np.mean(training_set, axis=0)
  #Centering the data
  Z_training = training_set - mean_vector 
  Z_testing = testing_set - mean_vector
  classify_vector=np.empty([test_data_size,1])
  kpca = KernelPCA(kernel='cosine', n_components = int(components))
  Z_training_kpca = kpca.fit_transform(Z_training)
  Z_testing_kpca = kpca.transform(Z_testing)
  euc_dist_KPCA = cdist(Z_testing_kpca,Z_training_kpca)
  classify_vector = knn(n_classes, training_labels, euc_dist_KPCA,1)
  y = np.count_nonzero(testing_labels-classify_vector)
  #images_display(test_data_size,testing_set,training_labels,classify_vector,training_set)
  cm = confusion_matrix(testing_labels, classify_vector, labels=confusion_labels)
  accuracy = (1-y/test_data_size)
  return accuracy, cm

##LDA

In [ ]:
def lda(D,label_vector, train_portion, n_classes, n_perclass, n_eig_used, k_knn, normal_split):
  # Splitting the dataset
  if normal_split ==False :
    training_set, testing_set, training_labels, testing_labels = split_nonfaces(D, label_vector)
  elif train_portion == 0.5:
    training_set, testing_set, training_labels, testing_labels = split_even_odd(D, label_vector)
  else:
    training_set, testing_set, training_labels, testing_labels = split(D, label_vector, train_portion, n_classes, n_perclass)  
  training_data_size = training_labels.shape[0]
  test_data_size = testing_labels.shape[0]
  n_trainperclass = int(train_portion * n_perclass)
  
  data_size = D.shape[0]
  feature_size = D.shape[1]
  #Mean Vector
  mean_vector = np.empty([n_classes, feature_size]) #mean vector for each class.
  for i in range(n_classes):
    mean_vector[i] = np.mean(training_set[i*n_trainperclass:(i*n_trainperclass)+n_trainperclass, :], axis=0)
  overall_mean = np.mean(mean_vector,keepdims=True, axis=0) #shape = 1x10304
  #Between class scatter matrix
  centered_means=mean_vector - overall_mean
  B = n_trainperclass*(centered_means.transpose()@centered_means) #shape = 10304x10304
  #Within class scatter matrix
  Z_training = np.empty([training_data_size, feature_size])
  for i in range(n_classes):
    Z_training[i*n_trainperclass:i*n_trainperclass+n_trainperclass, :] = training_set[i*n_trainperclass:i*n_trainperclass+n_trainperclass, :] - mean_vector[i]
  S = Z_training.transpose()@Z_training
  S_inv = np.linalg.inv(S)
  Matrix = S_inv@B
  #Eigen Values and Eigen Vectors
  eig_values, eig_vectors = np.linalg.eigh(Matrix)
  indices = eig_values.argsort()[::-1]  
  eig_values_vector = eig_values[indices]
  eig_values = np.diag(eig_values_vector)
  eig_vectors = eig_vectors[:,indices]
  #Projection
  P_LDA = eig_vectors[:,0:n_eig_used]
  Z_training_LDA = training_set - overall_mean
  projected_matrix_LDA = np.dot(Z_training_LDA, P_LDA)
  #Centering the testing data
  Z_testing_LDA = testing_set - overall_mean 
  projected_testing_matrix_LDA = np.dot(Z_testing_LDA, P_LDA)
  euc_dist_LDA = cdist(projected_testing_matrix_LDA, projected_matrix_LDA)
  #knn
  if isinstance(k_knn, (int, float)):
    k_knn=[k_knn]
  accuracy = np.empty([1, len(k_knn)])
  for kk in range(len(k_knn)):
    classify_vector = knn(n_classes, training_labels, euc_dist_LDA, k_knn[kk])
    y = np.count_nonzero(testing_labels-classify_vector)
    cm = confusion_matrix(testing_labels, classify_vector, labels=confusion_labels)
    accuracy[0,kk] = 1-y/test_data_size
  if n_classes == 2 or normal_split == False:
        for o in range(test_data_size):
          array = np.reshape(testing_set[o,:], (112,92))
          data = im.fromarray(array)
          plt.imshow(data)
          plt.axis('off')  
          plt.show()
          if classify_vector[o] == 1:
            print("This is a Non-face image")
          else:
            print("This is a face image")
 # else:
  #  images_display(test_data_size,testing_set,training_labels,classify_vector,training_set)
  return accuracy, cm

##QDA

In [ ]:
def qda(D,label_vector, train_portion, n_classes, n_perclass, reg_param, heatmap_bool):
# Splitting the dataset
  if train_portion == 0.5:
    training_set, testing_set, training_labels, testing_labels = split_even_odd(D, label_vector)
  else:
    training_set, testing_set, training_labels, testing_labels = split(D, label_vector, train_portion, n_classes, n_perclass)
  training_data_size = training_labels.shape[0]
  test_data_size = testing_labels.shape[0]
  n_trainperclass = int(train_portion * n_perclass)
  #Mean Vector 
  mean_vector = np.mean(training_set, axis=0)
  #Centering the data
  Z_training = training_set - mean_vector 
  Z_testing = testing_set - mean_vector
  classify_vector=np.empty([test_data_size,1])
  #QDA classifier fitting and transforming
  qda = QuadraticDiscriminantAnalysis(reg_param=reg_param)
  qda.fit(Z_training,  training_labels.ravel())
  classify_vector = np.empty([test_data_size,1])
  classify_vector[:,0] = qda.predict(Z_testing)
  y = np.count_nonzero(testing_labels-classify_vector)
  cm = confusion_matrix(testing_labels, classify_vector, labels=confusion_labels)
  accuracy = (1-y/test_data_size)
  if heatmap_bool:
    #Reduce number of samples in testing data to visualize the covariance matrix using heatmap
    Z_testing=Z_testing[1::20]
    corrmatrix = np.corrcoef(Z_testing, rowvar=False)
    corrmatrix = np.absolute(corrmatrix)
    corrmatrix = np.around(corrmatrix, decimals=4)
    # Plot heatmap
    sns.heatmap(corrmatrix, cmap='coolwarm', center=0, square=True)  
  return accuracy, cm

##Displaying Images

In [ ]:
def images_display(test_data_size,testing_set,training_labels,classify_vector,training_set):
  for o in range(test_data_size):
      fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
      array = np.reshape(testing_set[o,:], (112,92))
      data = im.fromarray(array)
      ax1.imshow(data)
      ax1.set_title('Testing Image')
      ax1.axis('off')
      indices = np.where(training_labels[:] == classify_vector[o])
      indices = np.array(indices)
      indices=indices[0,:]
      array1 = np.reshape(training_set[indices[0],:], (112,92))
      data1 = im.fromarray(array1)
      ax2.imshow(data1)
      ax2.set_title('Match')
      ax2.axis('off')
      plt.show()


#Generate the Data Matrix and the Label vector



##Data set of faces only

In [ ]:
###################################
data_size = 400
n_perclass = 10
number_of_classes = 40
n_eig_used = 39
reg_param = 0 
confusion_labels=[i for i in range(1, 41)]
###################################
folder_path = '/content/att-database-of-faces'
subdirs = [f.path for f in os.scandir(folder_path) if f.is_dir()]
data_matrix=np.empty([400, 10304])
label_vector=np.empty([400,1])
i = 0
for subdir, dirs, files in os.walk(folder_path):
  for filename in files:
    if filename != 'README':
      id = get_id(subdir)
      image_path=''+subdir+'/'+filename
      img = Image.open(image_path)
      label_vector[i] = id
      data_matrix[i] = np.array(img).flatten()
      i=i+1

##Data set of faces and non faces

In [ ]:
data_size = 800
n_perclass = 400
number_of_classes = 2
n_eig_used = 3
confusion_labels=[i for i in range(1, 3)]
reg_param = 1
data_matrix=np.empty([data_size , 10304])
# Set the directory containing the images you want to resize and convert
directory1 = '/content/facenonfacedata/face-vs-non-face/non-faces'
directory2 = '/content/facenonfacedata/face-vs-non-face/faces'
subdirs1 = [f.path for f in os.scandir(directory1) if f.is_dir()]
subdirs2 = [f.path for f in os.scandir(directory2) if f.is_dir()]
label_vector=np.empty([data_size,1])
# Set the target size for the resized images
target_size = (92, 112)
i=0
label_vector[0:400] = 1 #non faces
label_vector[400:data_size] = 2 #faces
for subdir1, dirs, files in os.walk(directory1):
  for filename in files:
      image_path=''+subdir1+'/'+filename
      img_non_face = cv2.imread(image_path)
      # Resize the image
      resized_img = cv2.resize(img_non_face, target_size)
      # Convert the image to grayscale
      gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)
      image_vector = gray_img.flatten()
      data_matrix[i] = image_vector
      i=i+1    
for subdir2, dirs, files in os.walk(directory2):
  for filename in files:
      image_path=''+subdir2+'/'+filename
      img = Image.open(image_path)
      data_matrix[i] = np.array(img).flatten()
      i=i+1

#Sample Runs

In [ ]:
#def pca(D,label_vector, alpha, train_portion, n_classes, n_perclass, k_knn):
#def lda(D,label_vector, train_portion, n_classes, n_perclass, n_eig_used, k_knn):
#def split_even_odd(data_matrix, label_vector):
#def split(data_matrix, label_vector, train_portion, n_classes, n_perclass):
#def knn (projected_testing_set, projected_training_set, number_of_classes, training_labels, euc_dist_matrix, k):
#def kernel_pca(D, label_vector, train_portion, components):


##PCA

In [ ]:
alpha = [0.8, 0.85, 0.9, 0.95]  
training_portion = 0.5
k = 1
if isinstance(k, int):
    k=[k]
if isinstance(alpha, (int,float)):
    alpha=[alpha]
accuracy, cm = pca(data_matrix, label_vector, alpha, training_portion, number_of_classes, n_perclass, k, True)
for i in range(len(alpha)):
  for j in range(len(k)):
    print("\tFor alpha = {}, the accuracy is {} % \n".format(alpha[i], accuracy[i,j]*100))

plt.stem(alpha, accuracy[:,0])
plt.xlabel('Alpha')
plt.ylabel('Accuracy')
plt.ylim(0.5, 1)
plt.title('PCA')
plt.show()
print("The confusion matrix is: \n", cm)


##Kernel PCA

In [ ]:
#K = 1
training_portion = 0.5
number_of_components = 10
accuracy, cm = kernel_pca(data_matrix, label_vector, training_portion, number_of_classes, n_perclass, number_of_components)
print("The accuracy is: {} %". format(accuracy * 100))
print("The confusion matrix is: \n", cm)
print(cm.shape)

## Kernel PCA vs PCA

In [ ]:
training_portion = 0.5
number_of_components = 10
explained_variance = 0.95
k = 1 

start_time = time.perf_counter()
accuracy_kpca, cm_kpca = kernel_pca(data_matrix, label_vector, training_portion, number_of_classes, n_perclass, number_of_components)
end_time = time.perf_counter()
execution_time_kpca = end_time - start_time
print("Kernel PCA execution time: {:.5f} seconds".format(execution_time_kpca))
print("The accuracy obtained is: ", accuracy_kpca * 100)
print("The confusion matrix is: \n", cm_kpca)

start_time = time.perf_counter()
accuracy, cm = pca(data_matrix, label_vector, explained_variance, training_portion, number_of_classes, n_perclass, k, True)
end_time = time.perf_counter()
execution_time = end_time - start_time
print("\n\nPCA execution time: {:.5f} seconds".format(execution_time))
print("The accuracy received is:  {} %". format(accuracy[0,0] * 100))
print("The confusion matrix is: \n", cm)

##PCA using different split ratios (70-30 & 20-80)

---



In [ ]:
#PCA
alpha = [0.95]  
k = [1]
print("for the 70 - 30 split:\n") 
training_portion = 0.7
accuracy, cm = pca(data_matrix, label_vector, alpha, training_portion, number_of_classes, n_perclass, k,True)
print("PCA: for alpha = {}, for k = {}, the accuracy is {} % \n".format(alpha[0], k[0], accuracy[0,0]*100))
#LDA
accuracy, cm = lda(data_matrix, label_vector, training_portion, number_of_classes, n_perclass, n_eig_used, k, True)
print("LDA: for k = {}, the accuracy is {} % \n".format(k[0], accuracy[0, 0]*100))



In [ ]:
print("for the 20 - 80 split:\n") 
training_portion = 0.2
accuracy, cm = pca(data_matrix, label_vector, alpha, training_portion, number_of_classes, n_perclass, k,True)
print("PCA: for alpha = {}, for k = {}, the accuracy is {} % \n".format(alpha[0], k[0], accuracy[0,0]*100))
#LDA
accuracy, cm = lda(data_matrix, label_vector, training_portion, number_of_classes, n_perclass, n_eig_used, k, True)
print("LDA: for k = {}, the accuracy is {} % \n".format(k[0], accuracy[0, 0]*100))

##LDA

In [ ]:
#def lda(D,label_vector, train_portion, n_classes, n_perclass, n_eig_used, k_knn):
train_portion = 0.5
n_classes = 40
n_perclass = 10
n_eig_used = 39
k = 1
accuracy, cm = lda(data_matrix, label_vector, train_portion, n_classes, n_perclass, n_eig_used, k, True)
if isinstance(k, int):
    k=[k]
for i in range(len(k)):
  print("\tFor k = {}, the accuracy is {} % \n".format(k[i], accuracy[0, i]*100))
print("The confusion matrix is: \n", cm)

##QDA

In [ ]:
training_portion = 0.5
heatmap_bool = True
accuracy, cm = qda(data_matrix, label_vector, training_portion, number_of_classes, n_perclass, reg_param, heatmap_bool)
print("The accuracy is: {} %". format(accuracy * 100))
print("The confusion matrix is: \n", cm)

##QDA vs LDA

In [ ]:
training_portion = 0.5
k = 1 
heatmap_bool = False

start_time = time.perf_counter()
accuracy_qda, cm_qda = qda(data_matrix, label_vector, training_portion, number_of_classes, n_perclass, reg_param, heatmap_bool)
end_time = time.perf_counter()
execution_time_qda = end_time - start_time
print("\n\nQDA execution time: {:.5f} seconds".format(execution_time_qda))
print("The accuracy is: {} %". format(accuracy_qda * 100))
print("The confusion matrix is: \n", cm_qda)


start_time = time.perf_counter()
accuracy, cm = lda(data_matrix, label_vector, training_portion, number_of_classes, n_perclass, n_eig_used,k, True)
end_time = time.perf_counter()
execution_time = end_time - start_time
print("\n\nLDA execution time: {:.5f} seconds".format(execution_time))
print("The accuracy is:  {} %". format(accuracy[0,0] * 100))
print("The confusion matrix is: \n", cm)

##PCA vs LDA (different k values)

In [ ]:
alpha = [0.95]  
training_portion = 0.5
k = [1, 3, 5, 7]
print('PCA: \n')
accuracy, cm = pca(data_matrix, label_vector, alpha, training_portion, number_of_classes, n_perclass, k, True)
for i in range(len(k)):
  print("For k = {}, the accuracy is {} % \n".format(k[i],  accuracy[0,i]*100))
plt.stem(k, accuracy[0,:])
plt.xlabel('k')
plt.ylabel('Accuracy')
plt.ylim(0.5, 1)
plt.title('PCA')
plt.show()


accuracy2, cm2 = lda(data_matrix, label_vector, training_portion, number_of_classes, n_perclass, n_eig_used, k, True)
print('LDA: \n')
for i in range(len(k)):
  print("\tFor k = {}, the accuracy is {} % \n".format(k[i], accuracy2[0, i]*100))
plt.stem(k, accuracy2[0,:])
plt.xlabel('k')
plt.ylabel('Accuracy')
plt.ylim(0.5, 1)
plt.title('LDA')
plt.show()


## Faces vs non-faces problem using PCA


In [ ]:
k_knn=1
alpha = 0.8
training_portion=0.5
accuracy, cm = pca(data_matrix, label_vector, alpha, training_portion, 2, 400, k_knn, True)
print("The accuracy received is:  {} %". format(accuracy[0,0] * 100))
print('Confusion Matrix: \n', cm)

## Faces vs non-faces problem using LDA


In [ ]:
train_portion=0.5
accuracy, cm = lda(data_matrix,label_vector, train_portion, number_of_classes, n_perclass, n_eig_used, 1, True)
print("The accuracy received is:  {} %". format(accuracy[0,0] * 100))
print("The confusion matrix:\n", cm)

##Variable number of non faces

In [ ]:
#Loop faces and nonfaces with variable nonfaces
#100, 200, 300 and 400 nonface images in the dataset
#50-50 split to be used with k=1 only

nonface_data_size= [100, 200, 300, 400] 
# at each time testing samples are fixed to 50 while traning samples changes through 50,100,150,200
explained_variance = 0.95
training_portion = 0.5
k = 1
shuffled_data_matrix=np.empty([2*n_perclass , 10304])
shuffled_data_matrix[0:n_perclass,:] = np.random.permutation(data_matrix[0:n_perclass,:])
shuffled_data_matrix[n_perclass:,:] = np.random.permutation(data_matrix[n_perclass :,:])
print(shuffled_data_matrix.shape)
accuracy = [0.0, 0.0, 0.0, 0.0]
for i in range(len(nonface_data_size)):
  #Initialisation of new data matrix and label vector
  new_sample_size = n_perclass + nonface_data_size[i]
  new_data_matrix=np.empty([new_sample_size , 10304])
  new_label_vector=np.empty([new_sample_size,1])
  #Data and labels splitting
  new_data_matrix[0:nonface_data_size[i],:]=shuffled_data_matrix[0:nonface_data_size[i],:]
  new_label_vector[0:nonface_data_size[i],:]=label_vector[0:nonface_data_size[i],:]
  new_data_matrix[nonface_data_size[i]:n_perclass+nonface_data_size[i],:]=shuffled_data_matrix[n_perclass:data_size,:]
  new_label_vector[nonface_data_size[i]:n_perclass+nonface_data_size[i],:]=label_vector[n_perclass:data_size,:]
  accuracy_received, cm = pca(new_data_matrix, new_label_vector, explained_variance, training_portion, number_of_classes, n_perclass, k, False)
  accuracy[i] = accuracy_received[0,0]

print("Number of faces is fixed to 400 samples\n")
print("Number of non-faces training: ")
for i in range(len(nonface_data_size)):
      print("\tFor {} non face training samples, the accuracy is {} % \n".format(nonface_data_size[i]-(50*(i+1)), accuracy[i]*100))
print("Note: Testing samples are fixed to 50\n")
for c in range(len(nonface_data_size)):
    nonface_data_size[c] -= 50
plt.stem(nonface_data_size, accuracy)
plt.xlabel('Nonface_data_size')
plt.ylabel('Accuracy')
plt.ylim(0.5, 1)
plt.title('Varying Nonface data size')
plt.show()
